In [2]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import torch

g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
# Equivalently, PyTorch LongTensors also work.
g = dgl.graph(
    (torch.LongTensor([0, 0, 0, 0, 0]), torch.LongTensor([1, 2, 3, 4, 5])),
    num_nodes=6,
)

# You can omit the number of nodes argument if you can tell the number of nodes from the edge list alone.
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]))
g.edges()

(tensor([0, 0, 0, 0, 0]), tensor([1, 2, 3, 4, 5]))

In [4]:
# Assign a 3-dimensional node feature vector for each node.
g.ndata["x"] = torch.randn(6, 3)
# Assign a 4-dimensional edge feature vector for each edge.
g.edata["a"] = torch.randn(5, 4)
# Assign a 5x4 node feature matrix for each node.  Node and edge features in DGL can be multi-dimensional.
g.ndata["y"] = torch.randn(6, 5, 4)

print(g.edata["a"])
print(g.ndata["x"])
print(g.ndata["y"])


tensor([[-0.2515, -0.6148, -0.8321, -0.4604],
        [ 0.9227,  1.2012,  1.1728,  1.2815],
        [ 0.6853,  0.4380, -0.3704,  0.0036],
        [-0.0367, -0.2028,  1.6394, -0.8001],
        [-1.2440,  0.1641, -1.3086,  0.7616]])
tensor([[-0.1655, -0.7565,  0.4053],
        [-0.5343, -1.3291,  0.0520],
        [ 0.0156, -1.0735, -0.6773],
        [-0.7313, -0.3863,  1.0811],
        [-1.1074, -1.4736, -1.3848],
        [-1.8301, -1.3398, -2.0094]])
tensor([[[ 5.8819e-01,  1.5830e+00,  4.1085e-01, -2.8881e-01],
         [ 6.8007e-01,  9.2380e-01,  1.3031e+00, -3.0698e-01],
         [ 1.4873e-01,  7.4959e-01,  5.3189e-01, -7.6768e-01],
         [-6.3639e-01,  1.3513e+00,  3.1591e-01,  5.9026e-01],
         [-8.8365e-01,  1.8307e-01,  1.2949e-01,  5.9148e-01]],

        [[ 7.8589e-02, -3.4953e-02,  1.0418e+00, -9.9710e-01],
         [-2.1596e-01, -1.4361e+00, -1.4294e-01,  4.2905e-01],
         [-3.0345e-01, -3.8175e-01,  3.4978e-01, -8.5302e-01],
         [ 1.1448e+00,  9.1974e-01,  2.5

In [5]:
print(g.num_nodes())
print(g.num_edges())
# Out degrees of the center node
print(g.out_degrees(0))
# In degrees of the center node - note that the graph is directed so the in degree should be 0.
print(g.in_degrees(0))

6
5
5
0


In [7]:
# Induce a subgraph from node 0, node 1 and node 3 from the original graph.
sg1 = g.subgraph([0, 1, 3])
# Induce a subgraph from edge 0, edge 1 and edge 3 from the original graph.
sg2 = g.edge_subgraph([0, 1, 3])
sg1,sg2

(Graph(num_nodes=3, num_edges=2,
       ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5, 4), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}),
 Graph(num_nodes=4, num_edges=3,
       ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5, 4), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}))

In [8]:
# The original IDs of each node in sg1
print(sg1.ndata[dgl.NID])
# The original IDs of each edge in sg1
print(sg1.edata[dgl.EID])
# The original IDs of each node in sg2
print(sg2.ndata[dgl.NID])
# The original IDs of each edge in sg2
print(sg2.edata[dgl.EID])

tensor([0, 1, 3])
tensor([0, 2])
tensor([0, 1, 2, 4])
tensor([0, 1, 3])


In [9]:
# The original node feature of each node in sg1
print(sg1.ndata["x"])
# The original edge feature of each node in sg1
print(sg1.edata["a"])
# The original node feature of each node in sg2
print(sg2.ndata["x"])
# The original edge feature of each node in sg2
print(sg2.edata["a"])

tensor([[-0.1655, -0.7565,  0.4053],
        [-0.5343, -1.3291,  0.0520],
        [-0.7313, -0.3863,  1.0811]])
tensor([[-0.2515, -0.6148, -0.8321, -0.4604],
        [ 0.6853,  0.4380, -0.3704,  0.0036]])
tensor([[-0.1655, -0.7565,  0.4053],
        [-0.5343, -1.3291,  0.0520],
        [ 0.0156, -1.0735, -0.6773],
        [-1.1074, -1.4736, -1.3848]])
tensor([[-0.2515, -0.6148, -0.8321, -0.4604],
        [ 0.9227,  1.2012,  1.1728,  1.2815],
        [-0.0367, -0.2028,  1.6394, -0.8001]])


In [10]:
newg = dgl.add_reverse_edges(g)
print(newg.edges())

(tensor([0, 0, 0, 0, 0, 1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5, 0, 0, 0, 0, 0]))


In [11]:
# Save graphs
dgl.save_graphs("graph.dgl", g)
dgl.save_graphs("graphs.dgl", [g, sg1, sg2])

# Load graphs
(g,), _ = dgl.load_graphs("graph.dgl")
print(g)
(g, sg1, sg2), _ = dgl.load_graphs("graphs.dgl")
print(g)
print(sg1)
print(sg2)


Graph(num_nodes=6, num_edges=5,
      ndata_schemes={'y': Scheme(shape=(5, 4), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32)})
Graph(num_nodes=6, num_edges=5,
      ndata_schemes={'y': Scheme(shape=(5, 4), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32)})
Graph(num_nodes=3, num_edges=2,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'y': Scheme(shape=(5, 4), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'a': Scheme(shape=(4,), dtype=torch.float32)})
Graph(num_nodes=4, num_edges=3,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'y': Scheme(shape=(5, 4), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'a': Scheme